In [ ]:
import cobra
import pandas as pd
cobra_config = cobra.Configuration()
cobra_config.solver = "gurobi"
obj = "bio1_biomass"
ModelDir = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM_OB3b/model/'

In [ ]:
md = cobra.io.read_sbml_model(ModelDir + 'GEM_ADVE_v9.xml')

medium = md.medium
for n in medium:
    medium[n] = 0
medium['EX_cpd01024_e0'] = 15 # Methane
medium['EX_cpd00007_e0'] = 1000 # O2
medium['EX_cpd00048_e0'] = 1000 # Sulfate
medium['EX_cpd00067_e0'] = 1000 # H+
medium['EX_cpd00209_e0'] = 1000 # Nitrate
medium['EX_cpd00009_e0'] = 1000 # Phsphate
medium['EX_cpd10516_e0'] = 1000 # Fe3+
medium['EX_cpd00030_e0'] = 1000 # Mn2+
medium['EX_cpd00034_e0'] = 1000 # Zn2+
medium['EX_cpd00058_e0'] = 1000 # Cu2+
medium['EX_cpd00063_e0'] = 1000 # Ca2+
medium['EX_cpd00099_e0'] = 1000 # Cl-
medium['EX_cpd00149_e0'] = 1000 # Cobalt
medium['EX_cpd00205_e0'] = 1000 # K+
medium['EX_cpd00254_e0'] = 1000 # Mg
#medium['EX_cpd00971_e0'] = 1000 # Na+
medium['EX_cpd03523_e0'] = 1000 # Dihydromethanopterin (H2MPT)

md.medium = medium


##### Methane assimilation pathway: methane -> methanol -> formaldehyde -> formate -> CO2

# Change proton to be pumped out to periplasm from cytoplasm while methanol oxidation
md.reactions.rxn14207_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00067_c0'): -2.0,
    md.metabolites.get_by_id('cpd00067_e0'): 2.0
})

# H2MPT transport reaction: transporting H2MPT from extracellular to cytosol
H2MPT_transport = cobra.Reaction("H2MPT_transport")
H2MPT_transport.name = 'H2MPT_transport'
H2MPT_transport.lower_bound = -1000
H2MPT_transport.upper_bound = 1000
H2MPT_transport.add_metabolites({
    md.metabolites.get_by_id('cpd03523_e0'): -1.0,
    md.metabolites.get_by_id('cpd03523_c0'): 1.0
})
md.add_reactions([H2MPT_transport])

# Add H4MPT as a reactant for biomass objective function. Coefficient of 0.00033 was adopted from Naizabekov and Lee (2020)
md.reactions.bio1_biomass.add_metabolites({"cpd00895_c0": -0.00033})

# Assume formaldehyde -> methylene-H4MPT to be irreversible reaction
md.reactions.rxn07847_c0.bounds = (0,1000)

# Assume formate-THF ligase (ADP-forming) to be irreversible: cannot produce ATP
md.reactions.rxn00690_c0.bounds = (0, 1000)

# Correct GPR for soluble methane monooxygenase: mmoXYBZDC
md.reactions.rxn00843_c0.gene_reaction_rule = 'ADVE02_v2_12510 and ADVE02_v2_12511 and ADVE02_v2_12512 and ADVE02_v2_12513 and ADVE02_v2_12514'

# Correct GPR for methanol dehydrogenase: mxaFI
md.reactions.rxn14207_c0.gene_reaction_rule = 'ADVE02_v2_12109 and ADVE02_v2_12107'

# Correct GPR for formaldehyde activating enzyme: (Fae1 and Fae2) or 13185
md.reactions.rxn07847_c0.gene_reaction_rule = '(ADVE02_v2_12301 and ADVE02_v2_12302) or ADVE02_v2_13185'

# Correct GPR for methylene tetrahydromethanopterin dehydrogenase (MtdA or MtdB)
md.reactions.rxn07848_c0.gene_reaction_rule = 'ADVE02_v2_12229 or ADVE02_v2_12565'

# Correct GPR for FhcABCD: formyl-H4MPT -> formylmethanofuran -> formate
md.reactions.rxn02431_c0.gene_reaction_rule = 'ADVE02_v2_14361 and ADVE02_v2_14362 and ADVE02_v2_14359 and ADVE02_v2_14360'
md.reactions.rxn07849_c0.gene_reaction_rule = 'ADVE02_v2_14361 and ADVE02_v2_14362 and ADVE02_v2_14359 and ADVE02_v2_14360'



##### Serine cycle
# Correct GPR for serine-glyoxylate aminotransferase
md.reactions.rxn00424_c0.gene_reaction_rule = 'ADVE02_v2_12567'
md.reactions.rxn00251_c0.bounds = (-1000, 0)


# Correct GPR for malate thiokinase: small subunit and large subunit
md.reactions.rxn00934_c0.gene_reaction_rule = 'ADVE02_v2_12562 and ADVE02_v2_12563'

# Assume production of glycerate from hydroxypyruvate to be irreversible
md.reactions.rxn01011_c0.bounds = (-1000, 0)
md.reactions.rxn01013_c0.bounds = (-1000, 0)


##### TCA cycle
# Correct GPR for isocitrate oxidoreductase
md.reactions.rxn00198_c0.gene_reaction_rule = 'ADVE02_v2_10351'

# Assume production of succinyl-CoA from AKG to be irreversible
md.reactions.rxn00441_c0.bounds = (0, 1000)
md.reactions.rxn02376_c0.bounds = (0, 1000)
md.reactions.rxn01872_c0.bounds = (-1000, 0)

# Assume production of isocitrate from citrate to be irreversible
md.reactions.rxn00974_c0.bounds = (0, 1000)
md.reactions.rxn01388_c0.bounds = (-1000, 0)



##### EMC cycle

# Correct GPR and product of rxn01451_c0, and assume to be irreversible
md.reactions.rxn01451_c0.gene_reaction_rule = 'ADVE02_v2_11873'
md.reactions.rxn01451_c0.add_metabolites({md.metabolites.get_by_id('cpd00842_c0'): 1.0, 
                                          md.metabolites.get_by_id('cpd02234_c0'): -1.0})
md.reactions.rxn01451_c0.bounds = (-1000, 0)

# Correct GPR for rxn01453_c0 and assume to be irreversible
md.reactions.rxn01453_c0.gene_reaction_rule = 'ADVE02_v2_10866 or ADVE02_v2_11389'
md.reactions.rxn01453_c0.bounds = (-1000, 0)

# Correct GPR for rxn02345_c0
md.reactions.rxn02345_c0.gene_reaction_rule = 'ADVE02_v2_10747'

# Correct GPR for two MclA: Glyoxalate + Propionyl-CoA <=> L-erythro-3-Methylmalyl-CoA
md.reactions.rxn00682_c0.gene_reaction_rule = 'ADVE02_v2_11719 or ADVE02_v2_12560'

# Correct GPR for mesaconyl-CoA hydratase (meaC): L-erythro-3-Methylmalyl-CoA <=> Mesaconyl-CoA
md.reactions.rxn03440_c0.gene_reaction_rule = 'ADVE02_v2_14344'

# Correct GPR for methylsuccinyl-CoA dehydrogenase (predicted by Erb el al., 2007): (2S)-Methylsuccinyl-CoA <=> Mesaconyl-CoA
md.reactions.rxn25269_c0.gene_reaction_rule = 'ADVE02_v2_11687'

# Correct GPR for ethylmalonyl-CoA mutase: (2R)-Ethylmalonyl-CoA <=> (2S)-Methylsuccinyl-CoA
md.reactions.rxn16151_c0.gene_reaction_rule = 'ADVE02_v2_10884'

# Correct GPR for ethylmalonyl-CoA epimerase: (2S)-Ethylmalonyl-CoA <=> (2R)-Ethylmalonyl-CoA
md.reactions.rxn16825_c0.gene_reaction_rule = 'ADVE02_v2_14064'

# Correct GPR for crotonyl-CoA carboxylase (ccr): Ethylmalonyl-CoA <=> Crotonyl-CoA
md.reactions.rxn16150_c0.gene_reaction_rule = 'ADVE02_v2_14369'

# Correct GPR for Propanoyl-CoA:CO2 ligase: propionyl-CoA <=> Methylmalonyl-CoA
md.reactions.rxn01355_c0.gene_reaction_rule = 'ADVE02_v2_10751 and ADVE02_v2_11744'




##### Glycolysis/gluconeogenesis

# Assume phosphofructokinase to be irreversible reaction
md.reactions.rxn00545_c0.bounds = (0, 1000)



##### Pyruvate Metabolism

# Assume reversiblities of three reactions for pyruvate <-> phosphoenolpyruvate as follows
md.reactions.rxn00147_c0.bounds = (0, 1000) # phosphoenolpyruvate synthase, PPS
md.reactions.rxn00151_c0.bounds = (0, 1000) # pyruvate phosphate dikinase, PPDK
md.reactions.rxn00148_c0.bounds = (-1000, 1000) # pyruvate kinase, PYK

# Assume production of acetyl-CoA from pyruvate to be irreversible, and correct GPR
md.reactions.rxn02342_c0.bounds = (0, 1000)
md.reactions.rxn01871_c0.bounds = (-1000, 0)
md.reactions.rxn02342_c0.gene_reaction_rule = '(ADVE02_v2_13741 and ADVE02_v2_13743) or (ADVE02_v2_10248 and ADVE02_v2_10246)'
md.reactions.rxn01871_c0.gene_reaction_rule = 'ADVE02_v2_10249 and ADVE02_v2_13740'

# Correct PGR for pyruvate decarboxylase
md.reactions.rxn00011_c0.gene_reaction_rule = '(ADVE02_v2_13741 and ADVE02_v2_13743) or (ADVE02_v2_10248 and ADVE02_v2_10246)'


##### Arginine and L-proline metabolism

# Correct the reversibility of reaction converting proline to 1-Pyrroline-5-carboxylate
md.reactions.rxn09188_c0.bounds = (0, 1000)

# Acetylornithine aminotransferase
md.reactions.rxn01637_c0.gene_reaction_rule = 'ADVE02_v2_12390 or ADVE02_v2_11278'
md.reactions.rxn00467_c0.gene_reaction_rule = 'ADVE02_v2_10107 or ADVE02_v2_11356'


##### Nitrogen assimilation
# Add NADPH-dependent nitrate reductase
rxn00572_c0 = cobra.Reaction("rxn00572_c0")
rxn00572_c0.name = 'NADPH-dependent nitrate reductase'
rxn00572_c0.lower_bound = 0
rxn00572_c0.upper_bound = 1000
rxn00572_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00209_c0'): -1.0,
    md.metabolites.get_by_id('cpd00067_c0'): -2.0,
    md.metabolites.get_by_id('cpd00005_c0'): -1.0,

    md.metabolites.get_by_id('cpd00075_c0'): 1.0,
    md.metabolites.get_by_id('cpd00006_c0'): 1.0,
    md.metabolites.get_by_id('cpd00001_c0'): 1.0
})
rxn00572_c0.gene_reaction_rule = 'ADVE02_v2_11064'
md.add_reactions([rxn00572_c0])



##### Aminopeptidase: assume no aminopeptidase reactions required in model simulation
# v12.reactions.rxn00350_c0.bounds = (0,0)
# v12.reactions.rxn00650_c0.bounds = (0,0)



##### Non-growth associated maintenance
NGAM_c0 = cobra.Reaction("NGAM_c0")
NGAM_c0.name = 'ATP required for non-growth associated maintenance'
NGAM_c0.lower_bound = 15
NGAM_c0.upper_bound = 15
NGAM_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00002_c0'): -1.0,
    md.metabolites.get_by_id('cpd00001_c0'): -1.0,

    md.metabolites.get_by_id('cpd00008_c0'): 1.0,
    md.metabolites.get_by_id('cpd00067_c0'): 1.0,
    md.metabolites.get_by_id('cpd00009_c0'): 1.0
})
md.add_reactions([NGAM_c0])


##### Electron transport chain for proton motive force

# Assume phosphate transporting ATPase to be irreversible
md.reactions.rxn05145_c0.bounds = (0, 1000)

### Complex I
# Assume NADH dehydrogenase to be irreversible
md.reactions.rxn10122_c0.bounds = (0, 1000)
# Correct GPR for NADH dehydrogenase
md.reactions.rxn10122_c0.gene_reaction_rule = 'ADVE02_v2_11206 and ADVE02_v2_14433 and ADVE02_v2_14047 and ADVE02_v2_14048 and ADVE02_v2_14050 and ADVE02_v2_14051 and ADVE02_v2_14052 and ADVE02_v2_14053 and ADVE02_v2_14054 and ADVE02_v2_14055 and ADVE02_v2_14056 and ADVE02_v2_14057 and ADVE02_v2_14058 and ADVE02_v2_14059 and ADVE02_v2_14060 and ADVE02_v2_14061'


### Complex II
# Assume Succinate dehydrogease reducing ubiquinone as a single-step reaction, and irreversible
md.reactions.rxn09272_c0.bounds = (0, 1000)


### Complex III
# Add cytochrome-c reductase
rxn13689_c0 = cobra.Reaction("rxn13689_c0")
rxn13689_c0.name = 'cytochrome-c reductase (ubiquinol8: 4 protons translocated)'
rxn13689_c0.lower_bound = 0
rxn13689_c0.upper_bound = 1000
rxn13689_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00109_c0'): -2.0,
    md.metabolites.get_by_id('cpd00067_c0'): -2.0,
    md.metabolites.get_by_id('cpd15561_c0'): -1.0,

    md.metabolites.get_by_id('cpd00110_c0'): 2.0,
    md.metabolites.get_by_id('cpd00067_e0'): 4.0,
    md.metabolites.get_by_id('cpd15560_c0'): 1.0
})
rxn13689_c0.gene_reaction_rule = 'ADVE02_v2_10640 and ADVE02_v2_10641 and ADVE02_v2_10642'
md.add_reactions([rxn13689_c0])


### Complex IV
## Hydrogenase reactions are assumed to have no flux 
# md.reactions.rxn10117_c0.bounds = (0,0)
# md.reactions.rxn10118_c0.bounds = (0,0)
# md.reactions.rxn10119_c0.bounds = (0,0)

# Correct GPR for cytochrome oxidase bo3
md.reactions.rxn10113_c0.gene_reaction_rule = '(ADVE02_v2_13970 and ADVE02_v2_13971 and ADVE02_v2_13972 and ADVE02_v2_13973) or (ADVE02_v2_12693 and ADVE02_v2_12694)'



##### Glyoxylate and dicarboxylate metabolism
# Correct GPR for NAD(P)H dependent glycolate/glyoxylate oxidoreductase
md.reactions.rxn00324_c0.gene_reaction_rule = 'ADVE02_v2_14254 and ADVE02_v2_14255 and ADVE02_v2_14256 and ADVE02_v2_13737'
md.reactions.rxn00512_c0.gene_reaction_rule = 'ADVE02_v2_14254 and ADVE02_v2_14255 and ADVE02_v2_14256 and ADVE02_v2_13737'

# Add glutamate dehydrogenase: converting AKG (2-oxoglutarate) to L-glutamate
rxn00182_c0 = cobra.Reaction("rxn00182_c0")
rxn00182_c0.name = 'Glutamate dehydrogenase-NADH'
rxn00182_c0.lower_bound = -1000
rxn00182_c0.upper_bound = 1000
rxn00182_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00024_c0'): -1.0,
    md.metabolites.get_by_id('cpd00013_c0'): -1.0,
    md.metabolites.get_by_id('cpd00004_c0'): -1.0,
    md.metabolites.get_by_id('cpd00067_c0'): -1.0,

    md.metabolites.get_by_id('cpd00023_c0'): 1.0,
    md.metabolites.get_by_id('cpd00003_c0'): 1.0,
    md.metabolites.get_by_id('cpd00001_c0'): 1.0
})
rxn00182_c0.gene_reaction_rule = 'ADVE02_v2_13745'
md.add_reactions([rxn00182_c0])

rxn00184_c0 = cobra.Reaction("rxn00184_c0")
rxn00184_c0.name = 'Glutamate dehydrogenase-NADH'
rxn00184_c0.lower_bound = -1000
rxn00184_c0.upper_bound = 1000
rxn00184_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00024_c0'): -1.0,
    md.metabolites.get_by_id('cpd00013_c0'): -1.0,
    md.metabolites.get_by_id('cpd00005_c0'): -1.0,
    md.metabolites.get_by_id('cpd00067_c0'): -1.0,

    md.metabolites.get_by_id('cpd00023_c0'): 1.0,
    md.metabolites.get_by_id('cpd00006_c0'): 1.0,
    md.metabolites.get_by_id('cpd00001_c0'): 1.0
})
rxn00184_c0.gene_reaction_rule = 'ADVE02_v2_13745'
md.add_reactions([rxn00184_c0])

# Glutamate synthase use NADH (Murrell and Dalton, 1983)
# Remove NADPH/NADP and add NADH/NAD
md.reactions.rxn00085_c0.add_metabolites({'cpd00006_c0':1})
md.reactions.rxn00085_c0.add_metabolites({'cpd00005_c0':-1})
md.reactions.rxn00085_c0.add_metabolites({'cpd00003_c0':-1})
md.reactions.rxn00085_c0.add_metabolites({'cpd00004_c0':1})

##### Fatty acid Synthesis

# Correct GPR for Acetyl-CoA:CO2 ligase forming malonyl-CoA
md.reactions.rxn00533_c0.gene_reaction_rule = 'ADVE02_v2_14271 and ADVE02_v2_11889 and ADVE02_v2_12412 and ADVE02_v2_14116 and ADVE02_v2_14117'

# Gapfill
rxn05329_c0 = cobra.Reaction("rxn05329_c0")
rxn05329_c0.name = '(3R)-3-Hydroxybutanoyl-[acyl-carrier-protein] hydro-lyase'
rxn05329_c0.lower_bound = -1000
rxn05329_c0.upper_bound = 1000
rxn05329_c0.add_metabolites({
    md.metabolites.get_by_id('cpd11478_c0'): -1.0,

    md.metabolites.get_by_id('cpd00001_c0'): 1.0,
    md.metabolites.get_by_id('cpd11465_c0'): 1.0
})
#rxn05329_c0.gene_reaction_rule = ''
md.add_reactions([rxn05329_c0])

rxn05330_c0 = cobra.Reaction("rxn05330_c0")
rxn05330_c0.name = '(3R)-3-Hydroxyhexanoyl-[acyl-carrier-protein] hydro-lyase'
rxn05330_c0.lower_bound = -1000
rxn05330_c0.upper_bound = 1000
rxn05330_c0.add_metabolites({
    md.metabolites.get_by_id('cpd11479_c0'): -1.0,

    md.metabolites.get_by_id('cpd00001_c0'): 1.0,
    md.metabolites.get_by_id('cpd11473_c0'): 1.0
})
#rxn05330_c0.gene_reaction_rule = ''
md.add_reactions([rxn05330_c0])

rxn05334_c0 = cobra.Reaction("rxn05334_c0")
rxn05334_c0.name = '(3R)-3-Hydroxyoctanoyl-[acyl-carrier-protein] hydro-lyase'
rxn05334_c0.lower_bound = -1000
rxn05334_c0.upper_bound = 1000
rxn05334_c0.add_metabolites({
    md.metabolites.get_by_id('cpd11483_c0'): -1.0,

    md.metabolites.get_by_id('cpd00001_c0'): 1.0,
    md.metabolites.get_by_id('cpd11471_c0'): 1.0
})
#rxn05334_c0.gene_reaction_rule = ''
md.add_reactions([rxn05334_c0])

rxn05333_c0 = cobra.Reaction("rxn05333_c0")
rxn05333_c0.name = '(3R)-3-Hydroxydecanoyl-[acyl-carrier-protein] hydro-lyase'
rxn05333_c0.lower_bound = -1000
rxn05333_c0.upper_bound = 1000
rxn05333_c0.add_metabolites({
    md.metabolites.get_by_id('cpd11482_c0'): -1.0,

    md.metabolites.get_by_id('cpd00001_c0'): 1.0,
    md.metabolites.get_by_id('cpd00067_c0'): 1.0,
    md.metabolites.get_by_id('cpd11475_c0'): 1.0
})
#rxn05333_c0.gene_reaction_rule = ''
md.add_reactions([rxn05333_c0])

rxn05331_c0 = cobra.Reaction("rxn05331_c0")
rxn05331_c0.name = '(3R)-3-Hydroxydodecanoyl-[acyl-carrier-protein] hydro-lyase'
rxn05331_c0.lower_bound = -1000
rxn05331_c0.upper_bound = 1000
rxn05331_c0.add_metabolites({
    md.metabolites.get_by_id('cpd11480_c0'): -1.0,

    md.metabolites.get_by_id('cpd00001_c0'): 1.0,
    md.metabolites.get_by_id('cpd11469_c0'): 1.0
})
#rxn05331_c0.gene_reaction_rule = ''
md.add_reactions([rxn05331_c0])

rxn05335_c0 = cobra.Reaction("rxn05335_c0")
rxn05335_c0.name = '(3R)-3-Hydroxytetradecanoyl-[acyl-carrier-protein] hydro-lyase'
rxn05335_c0.lower_bound = -1000
rxn05335_c0.upper_bound = 1000
rxn05335_c0.add_metabolites({
    md.metabolites.get_by_id('cpd11484_c0'): -1.0,

    md.metabolites.get_by_id('cpd00001_c0'): 1.0,
    md.metabolites.get_by_id('cpd11467_c0'): 1.0
})
#rxn05335_c0.gene_reaction_rule = ''
md.add_reactions([rxn05335_c0])

rxn05332_c0 = cobra.Reaction("rxn05332_c0")
rxn05332_c0.name = '(3R)-3-Hydroxyhexadecanoyl-[acyl-carrier-protein] hydro-lyase'
rxn05332_c0.lower_bound = -1000
rxn05332_c0.upper_bound = 1000
rxn05332_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00067_c0'): -1.0,
    md.metabolites.get_by_id('cpd11481_c0'): -1.0,

    md.metabolites.get_by_id('cpd00001_c0'): 1.0,
    md.metabolites.get_by_id('cpd11477_c0'): 1.0
})
#rxn05332_c0.gene_reaction_rule = ''
md.add_reactions([rxn05332_c0])

rxn05462_c0 = cobra.Reaction("rxn05462_c0")
rxn05462_c0.name = '(3R)-3-Hydroxyoctadecanoyl-[acyl-carrier-protein] hydro-lyase'
rxn05462_c0.lower_bound = -1000
rxn05462_c0.upper_bound = 1000
rxn05462_c0.add_metabolites({
    md.metabolites.get_by_id('cpd11571_c0'): -1.0,

    md.metabolites.get_by_id('cpd00001_c0'): 1.0,
    md.metabolites.get_by_id('cpd11572_c0'): 1.0
})
#rxn05462_c0.gene_reaction_rule = ''
md.add_reactions([rxn05462_c0])

rxn05411_c0 = cobra.Reaction("rxn05411_c0")
rxn05411_c0.name = '4-methyl-3-hydroxy-pentanoyl-ACP hydro-lyase'
rxn05411_c0.lower_bound = -1000
rxn05411_c0.upper_bound = 1000
rxn05411_c0.add_metabolites({
    md.metabolites.get_by_id('cpd11547_c0'): -1.0,

    md.metabolites.get_by_id('cpd00001_c0'): 1.0,
    md.metabolites.get_by_id('cpd11548_c0'): 1.0
})
#rxn05411_c0.gene_reaction_rule = ''
md.add_reactions([rxn05411_c0])

rxn05415_c0 = cobra.Reaction("rxn05415_c0")
rxn05415_c0.name = '6-methyl-3-hydroxy-heptanoyl-ACP hydro-lyase'
rxn05415_c0.lower_bound = -1000
rxn05415_c0.upper_bound = 1000
rxn05415_c0.add_metabolites({
    md.metabolites.get_by_id('cpd11551_c0'): -1.0,

    md.metabolites.get_by_id('cpd00001_c0'): 1.0,
    md.metabolites.get_by_id('cpd11552_c0'): 1.0
})
#rxn05415_c0.gene_reaction_rule = ''
md.add_reactions([rxn05415_c0])

rxn05419_c0 = cobra.Reaction("rxn05419_c0")
rxn05419_c0.name = '8-methyl-3-hydroxy-nonanoyl-ACP hydro-lyase'
rxn05419_c0.lower_bound = -1000
rxn05419_c0.upper_bound = 1000
rxn05419_c0.add_metabolites({
    md.metabolites.get_by_id('cpd11555_c0'): -1.0,

    md.metabolites.get_by_id('cpd00001_c0'): 1.0,
    md.metabolites.get_by_id('cpd11556_c0'): 1.0
})
#rxn05419_c0.gene_reaction_rule = ''
md.add_reactions([rxn05419_c0])

rxn05423_c0 = cobra.Reaction("rxn05423_c0")
rxn05423_c0.name = '10-methyl-3-hydroxy-undecanoyl-ACP hydro-lyase'
rxn05423_c0.lower_bound = -1000
rxn05423_c0.upper_bound = 1000
rxn05423_c0.add_metabolites({
    md.metabolites.get_by_id('cpd11559_c0'): -1.0,

    md.metabolites.get_by_id('cpd00001_c0'): 1.0,
    md.metabolites.get_by_id('cpd11560_c0'): 1.0
})
#rxn05423_c0.gene_reaction_rule = ''
md.add_reactions([rxn05423_c0])

rxn05427_c0 = cobra.Reaction("rxn05427_c0")
rxn05427_c0.name = '12-methyl-3-hydroxy-tridecanoyl-ACP hydro-lyase'
rxn05427_c0.lower_bound = -1000
rxn05427_c0.upper_bound = 1000
rxn05427_c0.add_metabolites({
    md.metabolites.get_by_id('cpd11563_c0'): -1.0,

    md.metabolites.get_by_id('cpd00001_c0'): 1.0,
    md.metabolites.get_by_id('cpd11564_c0'): 1.0
})
#rxn05427_c0.gene_reaction_rule = ''
md.add_reactions([rxn05427_c0])

rxn05431_c0 = cobra.Reaction("rxn05431_c0")
rxn05431_c0.name = '14-methyl-3-hydroxy-pentadecanoyl-ACP hydro-lyase'
rxn05431_c0.lower_bound = -1000
rxn05431_c0.upper_bound = 1000
rxn05431_c0.add_metabolites({
    md.metabolites.get_by_id('cpd11567_c0'): -1.0,

    md.metabolites.get_by_id('cpd00001_c0'): 1.0,
    md.metabolites.get_by_id('cpd11568_c0'): 1.0
})
#rxn05431_c0.gene_reaction_rule = ''
md.add_reactions([rxn05431_c0])



##### 'Riboflavin Metabolism' Gap
rxn05039_c0 = cobra.Reaction("rxn05039_c0")
rxn05039_c0.name = '5-amino-6-(5-phospho-D-ribitylamino)uracil phosphohydrolase'
rxn05039_c0.lower_bound = -1000
rxn05039_c0.upper_bound = 1000
rxn05039_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00001_c0'): -1.0,
    md.metabolites.get_by_id('cpd02720_c0'): -1.0,

    md.metabolites.get_by_id('cpd00009_c0'): 1.0,
    md.metabolites.get_by_id('cpd02882_c0'): 1.0
})
#rxn05039_c0.gene_reaction_rule = ''
md.add_reactions([rxn05039_c0])

# 'Riboflavin Metabolism' Gap
rxn08458_c0 = cobra.Reaction("rxn08458_c0")
rxn08458_c0.name = 'FADH2:NAD+ oxidoreductase'
rxn08458_c0.lower_bound = 0
rxn08458_c0.upper_bound = 1000
rxn08458_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00004_c0'): -1.0,
    md.metabolites.get_by_id('cpd00015_c0'): -1.0,
    md.metabolites.get_by_id('cpd00067_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00003_c0'): 1.0,
    md.metabolites.get_by_id('cpd00982_c0'): 1.0
})
rxn08458_c0.gene_reaction_rule = 'ADVE02_v2_10851' 
md.add_reactions([rxn08458_c0])

rxn03974_c0 = cobra.Reaction("rxn03974_c0")
rxn03974_c0.name = 'FMNH2:NAD+ oxidoreductase'
rxn03974_c0.lower_bound = 0
rxn03974_c0.upper_bound = 1000
rxn03974_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00004_c0'): -1.0,
    md.metabolites.get_by_id('cpd00050_c0'): -1.0,
    md.metabolites.get_by_id('cpd00067_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00003_c0'): 1.0,
    md.metabolites.get_by_id('cpd01270_c0'): 1.0
})
rxn03974_c0.gene_reaction_rule = 'ADVE02_v2_10805 or ADVE02_v2_10952' 
md.add_reactions([rxn03974_c0])

rxn15951_c0 = cobra.Reaction("rxn15951_c0")
rxn15951_c0.name = '5,6-dimethylbenzimidazole synthase'
rxn15951_c0.lower_bound = 0
rxn15951_c0.upper_bound = 1000
rxn15951_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00007_c0'): -1.0,
    md.metabolites.get_by_id('cpd01270_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00001_c0'): 1.0,
    md.metabolites.get_by_id('cpd00236_c0'): 1.0,
    md.metabolites.get_by_id('cpd01997_c0'): 1.0    
})
rxn15951_c0.gene_reaction_rule = 'ADVE02_v2_10278' 
md.add_reactions([rxn15951_c0])



##### 'Thiamine Metabolism' Gap
cpd00204_c0 = cobra.Metabolite('cpd00204_c0', formula='CO', name='CO_c0', compartment='c0')
cpd00204_e0 = cobra.Metabolite('cpd00204_e0', formula='CO', name='CO_e0', compartment='e0')

rxn20643_c0 = cobra.Reaction("rxn20643_c0")
rxn20643_c0.name = '5-amino-1-(5-phospho-D-ribosyl)imidazole formate-lyase (decarboxylating, 4-amino-2-methyl-5-phosphomethylpyrimidine-forming)'
rxn20643_c0.lower_bound = -1000
rxn20643_c0.upper_bound = 1000
rxn20643_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00017_c0'): -1.0,
    md.metabolites.get_by_id('cpd02140_c0'): -1.0,

    md.metabolites.get_by_id('cpd00047_c0'): 1.0,
    md.metabolites.get_by_id('cpd00060_c0'): 1.0,
    md.metabolites.get_by_id('cpd00067_c0'): 1.0,
    cpd00204_c0: 1.0,
    md.metabolites.get_by_id('cpd02775_c0'): 1.0,
    md.metabolites.get_by_id('cpd03091_c0'): 1.0
})
rxn20643_c0.gene_reaction_rule = 'ADVE02_v2_14134'
md.add_reactions([rxn20643_c0])


# In order to use rxn20643 - added above - , 4-Hydroxy-benzylalcohol, 5'-Deoxyadenosine, and carbon monooxide need to be secreted outside the model
CO_exchange = cobra.Reaction("CO_exchange")
CO_exchange.name = 'CO_exchange'
CO_exchange.lower_bound = -1000
CO_exchange.upper_bound = 1000
CO_exchange.add_metabolites({
    cpd00204_c0: -1.0,

    cpd00204_e0: 1.0
})
md.add_reactions([CO_exchange])

CO_secretion = cobra.Reaction("CO_secretion")
CO_secretion.name = 'CO_secretion'
CO_secretion.lower_bound = 0
CO_secretion.upper_bound = 1000
CO_secretion.add_metabolites({
    cpd00204_e0: -1.0
})
md.add_reactions([CO_secretion])

cpd15378_e0 = cobra.Metabolite('cpd15378_e0', formula='C7H8O2', name='4-Hydroxy-benzylalcohol_e0', compartment='e0')
HBA_exchange = cobra.Reaction("HBA_exchange")
HBA_exchange.name = 'HBA_exchange'
HBA_exchange.lower_bound = -1000
HBA_exchange.upper_bound = 1000
HBA_exchange.add_metabolites({
    md.metabolites.get_by_id('cpd15378_c0'): -1.0,
    
    cpd15378_e0: 1.0,
})
md.add_reactions([HBA_exchange])

HBA_secretion = cobra.Reaction("HBA_secretion")
HBA_secretion.name = 'HBA_secretion'
HBA_secretion.lower_bound = 0
HBA_secretion.upper_bound = 1000
HBA_secretion.add_metabolites({
    cpd15378_e0: -1.0
})
md.add_reactions([HBA_secretion])





##### Porphyrin Metabolism
# Gapfil
cpd03760_c0 = cobra.Metabolite('cpd03760_c0', formula='C44H49N4O16', name='Precorrin 6B_c0', compartment='c0')
rxn03492_c0 = cobra.Reaction("rxn03492_c0")
rxn03492_c0.name = 'precorrin-6Y:NADP+ oxidoreductase'
rxn03492_c0.lower_bound = -1000
rxn03492_c0.upper_bound = 1000
rxn03492_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00005_c0'): -1.0,
    md.metabolites.get_by_id('cpd00067_c0'): -1.0,
    md.metabolites.get_by_id('cpd03761_c0'): -1.0,

    md.metabolites.get_by_id('cpd00006_c0'): 1.0,
    cpd03760_c0: 1.0
})
rxn03492_c0.gene_reaction_rule = 'ADVE02_v2_13112'
md.add_reactions([rxn03492_c0])

rxn03491_c0 = cobra.Reaction("rxn03491_c0")
rxn03491_c0.name = 'S-Adenosyl-L-methionine:1-precorrin-6Y C5,15-methyltransferase (C-12-decarboxylating)'
rxn03491_c0.lower_bound = -1000
rxn03491_c0.upper_bound = 1000
rxn03491_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00017_c0'): -2.0,
    cpd03760_c0: -1.0,
    
    md.metabolites.get_by_id('cpd00011_c0'): 1.0,
    md.metabolites.get_by_id('cpd00019_c0'): 2.0,
    md.metabolites.get_by_id('cpd00067_c0'): 2.0,
    md.metabolites.get_by_id('cpd03835_c0'): 1.0
})
rxn03491_c0.gene_reaction_rule = 'ADVE02_v2_13111'
md.add_reactions([rxn03491_c0])

rxn06887_c0 = cobra.Reaction("rxn06887_c0")
rxn06887_c0.name = 'R05224'
rxn06887_c0.lower_bound = -1000
rxn06887_c0.upper_bound = 1000
rxn06887_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00001_c0'): -2.0,
    md.metabolites.get_by_id('cpd00002_c0'): -2.0,
    md.metabolites.get_by_id('cpd00053_c0'): -2.0,
    md.metabolites.get_by_id('cpd03832_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00008_c0'): 2.0,
    md.metabolites.get_by_id('cpd00009_c0'): 2.0,
    md.metabolites.get_by_id('cpd00023_c0'): 2.0,
    md.metabolites.get_by_id('cpd00067_c0'): 2.0,
    md.metabolites.get_by_id('cpd03913_c0'): 1.0
})
rxn06887_c0.gene_reaction_rule = 'ADVE02_v2_13108'
md.add_reactions([rxn06887_c0])

rxn04386_c0 = cobra.Reaction("rxn04386_c0")
rxn04386_c0.name = 'ATP:L-threonine O-phosphotransferase'
rxn04386_c0.lower_bound = 0
rxn04386_c0.upper_bound = 1000
rxn04386_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00002_c0'): -1.0,
    md.metabolites.get_by_id('cpd00161_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00008_c0'): 1.0,
    md.metabolites.get_by_id('cpd00067_c0'): 1.0,
    md.metabolites.get_by_id('cpd08928_c0'): 1.0    
})
# rxn04386_c0.gene_reaction_rule = 'ADVE02_v2_' 
md.add_reactions([rxn04386_c0])

cpd02211_c0 = cobra.Metabolite('cpd02211_c0', formula='C4H7NO3', name='L-2-Amino-acetoacetate_c0', compartment='c0')
rxn00274_c0 = cobra.Reaction("rxn00274_c0")
rxn00274_c0.name = 'Acetyl-CoA:glycine C-acetyltransferase'
rxn00274_c0.lower_bound = -1000
rxn00274_c0.upper_bound = 1000
rxn00274_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00022_c0'): -1.0,
    md.metabolites.get_by_id('cpd00033_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00010_c0'): 1.0,
    cpd02211_c0: 1.0
})
rxn00274_c0.gene_reaction_rule = 'ADVE02_v2_12831' 
md.add_reactions([rxn00274_c0])

cpd01298_c0 = cobra.Metabolite('cpd01298_c0', formula='C3H8NO', name='Aminoacetone_c0', compartment='c0')
rxn02666_c0 = cobra.Reaction("rxn02666_c0")
rxn02666_c0.name = 'R03758'
rxn02666_c0.lower_bound = 0
rxn02666_c0.upper_bound = 1000
rxn02666_c0.add_metabolites({
    cpd02211_c0: -1.0,
    md.metabolites.get_by_id('cpd00067_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00011_c0'): 1.0,
    cpd01298_c0: 1.0
})
# rxn02666_c0.gene_reaction_rule = 'ADVE02_v2_' 
md.add_reactions([rxn02666_c0])

rxn02667_c0 = cobra.Reaction("rxn02667_c0")
rxn02667_c0.name = 'R03758'
rxn02667_c0.lower_bound = 0
rxn02667_c0.upper_bound = 1000
rxn02667_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00004_c0'): -1.0,
    md.metabolites.get_by_id('cpd00067_c0'): -1.0,
    cpd01298_c0: -1.0,
    
    md.metabolites.get_by_id('cpd00003_c0'): 1.0,
    md.metabolites.get_by_id('cpd02039_c0'): 1.0,
})
# rxn02667_c0.gene_reaction_rule = 'ADVE02_v2_' 
md.add_reactions([rxn02667_c0])

# Remove dimethylbenzimidazole from the products of biomass objective function
Dimethylbenzimidazole_coefficient = md.reactions.bio1_biomass.get_coefficient('cpd01997_c0')
md.reactions.bio1_biomass.add_metabolites({'cpd01997_c0':-Dimethylbenzimidazole_coefficient})

# Remove cobinamide from the products of biomass objective function
Cobinamide_coefficient = md.reactions.bio1_biomass.get_coefficient('cpd03422_c0')
md.reactions.bio1_biomass.add_metabolites({'cpd03422_c0':-Cobinamide_coefficient})

# Add heme O as a reactant for biomass objective function. Use same coefficient as heme.
heme_coefficient = md.reactions.bio1_biomass.get_coefficient('cpd00028_c0')
md.reactions.bio1_biomass.add_metabolites({'cpd11313_c0':heme_coefficient})



##### Folate Biosynthesis

# Assume the reversibilities of following reactions
md.reactions.rxn01603_c0.bounds = (0, 1000)
md.reactions.rxn03841_c0.bounds = (0, 1000)

# Correct reversibility
md.reactions.rxn03167_c0.bounds = (0, 1000)

# Gapfill
cpd02315_c0 = cobra.Metabolite('cpd02315_c0', formula='C9H11N5O3', name='6-Pyruvoyltetrahydropterin_c0', compartment='c0')
rxn02986_c0 = cobra.Reaction("rxn02986_c0")
rxn02986_c0.name = '2-amino-4-hydroxy-6-(erythro-1,2,3-trihydroxypropyl) dihydropteridine triphosphate lyase'
rxn02986_c0.lower_bound = -1000
rxn02986_c0.upper_bound = 1000
rxn02986_c0.add_metabolites({
    md.metabolites.get_by_id('cpd02978_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00421_c0'): 1.0,
    cpd02315_c0: 1.0
})
rxn02986_c0.gene_reaction_rule = 'ADVE02_v2_11384'
md.add_reactions([rxn02986_c0])

rxn11742_c0 = cobra.Reaction("rxn11742_c0")
rxn11742_c0.name = 'tetrahydrobiopterin:NADP+ oxidoreductase (6-pyruvoyltetrahydropterin-forming)'
rxn11742_c0.lower_bound = -1000
rxn11742_c0.upper_bound = 1000
rxn11742_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00005_c0'): -2.0,
    md.metabolites.get_by_id('cpd00067_c0'): -2.0,
    cpd02315_c0: -1.0,
    
    md.metabolites.get_by_id('cpd00006_c0'): 2.0,
    md.metabolites.get_by_id('cpd00233_c0'): 1.0
})
# rxn11742_c0.gene_reaction_rule = '' <- added for gapfill, no evidence found yet
md.add_reactions([rxn11742_c0])

cpd01895_c0 = cobra.Metabolite('cpd01895_c0', formula='C9H13N5O3', name='7,8-Dihydrobiopterin_c0', compartment='c0')
rxn47488_c0 = cobra.Reaction("rxn47488_c0")
rxn47488_c0.name = 'tetrahydrobiopterin:NADP+ oxidoreductase (6-pyruvoyltetrahydropterin-forming)'
rxn47488_c0.lower_bound = -1000
rxn47488_c0.upper_bound = 1000
rxn47488_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00005_c0'): -1.0,
    md.metabolites.get_by_id('cpd00067_c0'): -1.0,
    cpd01895_c0: -1.0,
    
    md.metabolites.get_by_id('cpd00006_c0'): 1.0,
    md.metabolites.get_by_id('cpd00233_c0'): 1.0
})
rxn47488_c0.gene_reaction_rule = 'ADVE02_v2_10856' 
md.add_reactions([rxn47488_c0])

rxn99999_c0 = cobra.Reaction("rxn99999_c0")
rxn99999_c0.name = 'R11767'
rxn99999_c0.lower_bound = -1000
rxn99999_c0.upper_bound = 1000
rxn99999_c0.add_metabolites({ 
    cpd01895_c0: -1.0,

    md.metabolites.get_by_id('cpd00231_c0'): 1.0
})
# rxn99999_c0.gene_reaction_rule = '' <- non-enzymatic
md.add_reactions([rxn99999_c0])


# # Glycoaldehyde secretion for tetrahydrofolate biosynthesis, 'cpd00229_c0'
# cpd00229_secretion = cobra.Reaction("cpd00229_secretion")
# cpd00229_secretion.name = 'cpd00229_secretion'
# cpd00229_secretion.lower_bound = 0
# cpd00229_secretion.upper_bound = 1000
# cpd00229_secretion.add_metabolites({
#     md.metabolites.get_by_id('cpd00229_c0'): -1.0
# })
# md.add_reactions([cpd00229_secretion])


rxn00979_c0 = cobra.Reaction("rxn00979_c0")
rxn00979_c0.name = 'glycolaldehyde:NAD+ oxidoreductase'
rxn00979_c0.lower_bound = -1000
rxn00979_c0.upper_bound = 1000
rxn00979_c0.add_metabolites({ 
    md.metabolites.get_by_id('cpd00001_c0'): -1.0,
    md.metabolites.get_by_id('cpd00003_c0'): -1.0,
    md.metabolites.get_by_id('cpd00229_c0'): -1.0,

    md.metabolites.get_by_id('cpd00004_c0'): 1.0,
    md.metabolites.get_by_id('cpd00067_c0'): 2.0,
    md.metabolites.get_by_id('cpd00139_c0'): 1.0
})
rxn00979_c0.gene_reaction_rule = 'ADVE02_v2_12055 or ADVE02_v2_12734'
md.add_reactions([rxn00979_c0])







##### Ubiquinon and Other Terpenoid-Quinone Biosynthesis
# Gapfill
rxn08130_c0 = cobra.Reaction("rxn08130_c0")
rxn08130_c0.name = 'S-adenosyl-L-methione:demethylmenaquinol methyltransferase'
rxn08130_c0.lower_bound = 0
rxn08130_c0.upper_bound = 1000
rxn08130_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00017_c0'): -1.0,
    md.metabolites.get_by_id('cpd15353_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00019_c0'): 1.0,
    md.metabolites.get_by_id('cpd00067_c0'): 1.0,
    md.metabolites.get_by_id('cpd15499_c0'): 1.0
})
rxn08130_c0.gene_reaction_rule = 'ADVE02_v2_11241' 
md.add_reactions([rxn08130_c0])

##### Lysine Biosynthesis
# Gapfill
rxn03087_c0 = cobra.Reaction("rxn03087_c0")
rxn03087_c0.name = 'N-Succinyl-L-2,6-diaminoheptanedioate:2-oxoglutarate amino-transferase'
rxn03087_c0.lower_bound = -1000
rxn03087_c0.upper_bound = 1000
rxn03087_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00024_c0'): -1.0,
    md.metabolites.get_by_id('cpd02698_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00023_c0'): 1.0,
    md.metabolites.get_by_id('cpd02724_c0'): 1.0
})
rxn03087_c0.gene_reaction_rule = 'ADVE02_v2_12390' 
md.add_reactions([rxn03087_c0])

# Correct reversibility
md.reactions.rxn03031_c0.bounds = (0, 1000)

##### Phenylalanine, Tyrosine and Tryptophan Biosynthesis
# Gapfill
rxn01739_c0 = cobra.Reaction("rxn01739_c0")
rxn01739_c0.name = 'Shikimate kinase'
rxn01739_c0.lower_bound = 0
rxn01739_c0.upper_bound = 1000
rxn01739_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00383_c0'): -1.0,
    md.metabolites.get_by_id('cpd00002_c0'): -1.0,

    md.metabolites.get_by_id('cpd02030_c0'): 1.0,
    md.metabolites.get_by_id('cpd00008_c0'): 1.0,
    md.metabolites.get_by_id('cpd00067_c0'): 1.0
})
rxn01739_c0.gene_reaction_rule = 'ADVE02_v2_13732'
md.add_reactions([rxn01739_c0])

rxn00490_c0 = cobra.Reaction("rxn00490_c0")
rxn00490_c0.name = 'L-arogenate hydro-lyase'
rxn00490_c0.lower_bound = 0
rxn00490_c0.upper_bound = 1000
rxn00490_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00616_c0'): -1.0,
    md.metabolites.get_by_id('cpd00067_c0'): -1.0,

    md.metabolites.get_by_id('cpd00066_c0'): 1.0,
    md.metabolites.get_by_id('cpd00011_c0'): 1.0,
    md.metabolites.get_by_id('cpd00001_c0'): 1.0
})
rxn00490_c0.gene_reaction_rule = 'ADVE02_v2_14209'
md.add_reactions([rxn00490_c0])

rxn00525_c0 = cobra.Reaction("rxn00525_c0")
rxn00525_c0.name = 'L-arogenate:NAD+ oxidoreductase'
rxn00525_c0.lower_bound = 0
rxn00525_c0.upper_bound = 1000
rxn00525_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00616_c0'): -1.0,
    md.metabolites.get_by_id('cpd00003_c0'): -1.0,

    md.metabolites.get_by_id('cpd00069_c0'): 1.0,
    md.metabolites.get_by_id('cpd00011_c0'): 1.0,
    md.metabolites.get_by_id('cpd00004_c0'): 1.0
})
rxn00525_c0.gene_reaction_rule = 'ADVE02_v2_13278'
md.add_reactions([rxn00525_c0])



##### Valine, Leucine and Isoleucine Biosynthesis
# Gapfill
rxn01208_c0 = cobra.Reaction("rxn01208_c0")
rxn01208_c0.name = '2-Oxo-4-methyl-3-carboxypentanoate decarboxylation'
rxn01208_c0.lower_bound = 0
rxn01208_c0.upper_bound = 1000
rxn01208_c0.add_metabolites({
    md.metabolites.get_by_id('cpd02605_c0'): -1.0,
    md.metabolites.get_by_id('cpd00067_c0'): -1.0,

    md.metabolites.get_by_id('cpd00200_c0'): 1.0,
    md.metabolites.get_by_id('cpd00011_c0'): 1.0
})
rxn01208_c0.gene_reaction_rule = 'ADVE02_v2_11144'
md.add_reactions([rxn01208_c0])


##### Valine, Leucine and Isoleucine Degradation
# Correct reversibility
md.reactions.rxn02866_c0.bounds = (-1000, 1000)

# Correct GPR
md.reactions.rxn01480_c0.gene_reaction_rule = 'ADVE02_v2_10838'

# Gapfil
cpd01468_c0 = cobra.Metabolite('cpd01468_c0', formula='C4H4O4', name='Methylmalonate_c0', compartment='c0')
rxn02729_c0 = cobra.Reaction("rxn02729_c0")
rxn02729_c0.name = '(S)-Methylmalonate semialdehyde:NAD+ oxidoreductase'
rxn02729_c0.lower_bound = -1000
rxn02729_c0.upper_bound = 1000
rxn02729_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00001_c0'): -1.0,
    md.metabolites.get_by_id('cpd00003_c0'): -1.0,
    md.metabolites.get_by_id('cpd00287_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00004_c0'): 1.0,
    md.metabolites.get_by_id('cpd00067_c0'): 2.0,
    cpd01468_c0: 1.0
})
rxn02729_c0.gene_reaction_rule = 'ADVE02_v2_11355' 
md.add_reactions([rxn02729_c0])


rxn39397_c0 = cobra.Reaction("rxn39397_c0")
rxn39397_c0.name = 'methylmalonate:CoA ligase (AMP-forming)'
rxn39397_c0.lower_bound = -1000
rxn39397_c0.upper_bound = 1000
rxn39397_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00002_c0'): -1.0,
    md.metabolites.get_by_id('cpd00010_c0'): -1.0,
    cpd01468_c0: -1.0,
    
    md.metabolites.get_by_id('cpd00012_c0'): 1.0,
    md.metabolites.get_by_id('cpd00018_c0'): 1.0,
    md.metabolites.get_by_id('cpd00067_c0'): 1.0,    
    md.metabolites.get_by_id('cpd00891_c0'): 1.0
})
rxn39397_c0.gene_reaction_rule = 'ADVE02_v2_11355' 
md.add_reactions([rxn39397_c0])

rxn01924_c0 = cobra.Reaction("rxn01924_c0")
rxn01924_c0.name = 'isobutyryl-CoA dehydrogenase'
rxn01924_c0.lower_bound = -1000
rxn01924_c0.upper_bound = 1000
rxn01924_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00015_c0'): -1.0,
    md.metabolites.get_by_id('cpd00481_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00982_c0'): 1.0,
    md.metabolites.get_by_id('cpd02187_c0'): 1.0
})
rxn01924_c0.gene_reaction_rule = 'ADVE02_v2_12409' 
md.add_reactions([rxn01924_c0])



##### Putrescine/Spermidine/Spermine/Pantothenate
# Spermidine_coefficient = md.reactions.bio1_biomass.get_coefficient('cpd00264_c0')
# md.reactions.bio1_biomass.add_metabolites({'cpd00264_c0':-Spermidine_coefficient})

# Correct GPR
md.reactions.rxn00470_c0.gene_reaction_rule = 'ADVE02_v2_13749'

# Correct reversibility
md.reactions.rxn02061_c0.bounds = (0, 1000)
md.reactions.rxn13388_c0.bounds = (0, 1000)


##### Cysteine and Methionine Metabolism
# Gapfill
rxn00127_c0 = cobra.Reaction("rxn00127_c0")
rxn00127_c0.name = 'S-adenosyl-L-methionine carboxy-lyase [(5-deoxy-5-adenosyl)(3-aminopropyl)methylsulfonium-forming]'
rxn00127_c0.lower_bound = 0
rxn00127_c0.upper_bound = 1000
rxn00127_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00017_c0'): -1.0,
    md.metabolites.get_by_id('cpd00067_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00011_c0'): 1.0,
    md.metabolites.get_by_id('cpd00837_c0'): 1.0
})
# rxn00127_c0.gene_reaction_rule = 'ADVE02_v2_' 
md.add_reactions([rxn00127_c0])

rxn05104_c0 = cobra.Reaction("rxn05104_c0")
rxn05104_c0.name = 'S-methyl-5-thio-D-ribulose-1-phosphate hydro-lyase'
rxn05104_c0.lower_bound = 0
rxn05104_c0.upper_bound = 1000
rxn05104_c0.add_metabolites({
    md.metabolites.get_by_id('cpd02791_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00001_c0'): 1.0,
    md.metabolites.get_by_id('cpd11295_c0'): 1.0
})
rxn05104_c0.gene_reaction_rule = 'ADVE02_v2_10781' 
md.add_reactions([rxn05104_c0])


cpd00869_c0 = cobra.Metabolite('cpd00869_c0', formula='C5H7O3S', name='4-Methylthio 2-oxobutyrate_c0', compartment='c0')
rxn05092_c0 = cobra.Reaction("rxn05092_c0")
rxn05092_c0.name = '1,2-dihydroxy-5-(methylthio)pent-1-en-3-one:oxygen oxidoreductase (formate-forming)'
rxn05092_c0.lower_bound = 0
rxn05092_c0.upper_bound = 1000
rxn05092_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00007_c0'): -1.0,
    md.metabolites.get_by_id('cpd11255_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00047_c0'): 1.0,
    md.metabolites.get_by_id('cpd00067_c0'): 2.0,
    cpd00869_c0: 1.0
})
# rxn05092_c0.gene_reaction_rule = 'ADVE02_v2_' 
md.add_reactions([rxn05092_c0])

rxn05108_c0 = cobra.Reaction("rxn05108_c0")
rxn05108_c0.name = 'R07396'
rxn05108_c0.lower_bound = 0
rxn05108_c0.upper_bound = 1000
rxn05108_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00023_c0'): -1.0,
    cpd00869_c0: -1.0,
    
    md.metabolites.get_by_id('cpd00024_c0'): 1.0,
    md.metabolites.get_by_id('cpd00060_c0'): 1.0
})
rxn05108_c0.gene_reaction_rule = 'ADVE02_v2_12831' 
md.add_reactions([rxn05108_c0])


##### Biotin Metabolism
# Gapfill
cpd01727_c0 = cobra.Metabolite('cpd01727_c0', formula='C7H10O4', name='Pimelate_c0', compartment='c0')
rxn10008_c0 = cobra.Reaction("rxn10008_c0")
rxn10008_c0.name = 'Lumped reaction for synthesis of pimelate'
rxn10008_c0.lower_bound = 0
rxn10008_c0.upper_bound = 1000
rxn10008_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00001_c0'): -2.0,
    md.metabolites.get_by_id('cpd00020_c0'): -1.0,
    md.metabolites.get_by_id('cpd00032_c0'): -1.0,
    md.metabolites.get_by_id('cpd00067_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00007_c0'): 3.0,
    cpd01727_c0: 1.0
})
# rxn10008_c0.gene_reaction_rule = 'ADVE02_v2_' 
md.add_reactions([rxn10008_c0])

rxn02296_c0 = cobra.Reaction("rxn02296_c0")
rxn02296_c0.name = '6-Carboxyhyxanoate:CoA ligase (AMP-forming)'
rxn02296_c0.lower_bound = 0
rxn02296_c0.upper_bound = 1000
rxn02296_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00002_c0'): -1.0,
    md.metabolites.get_by_id('cpd00010_c0'): -1.0,
    cpd01727_c0: -1.0,
    
    md.metabolites.get_by_id('cpd00012_c0'): 1.0,
    md.metabolites.get_by_id('cpd00018_c0'): 1.0,
    md.metabolites.get_by_id('cpd00067_c0'): 1.0,
    md.metabolites.get_by_id('cpd00782_c0'): 1.0
})
# rxn02296_c0.gene_reaction_rule = 'ADVE02_v2_' 
md.add_reactions([rxn02296_c0])

# Change biotin synthase to use sulfide, not elemental sulfur
md.reactions.rxn08180_c0.add_metabolites({md.metabolites.get_by_id('cpd00074_c0'): 1.0, 
                                          md.metabolites.get_by_id('cpd00239_c0'): -1.0,
                                          md.metabolites.get_by_id('cpd00067_c0'): 2.0})

# Add biotin as a reactant for biomass objective function. Use same coefficient as pyridoxal phosphate
biotin_coefficient = md.reactions.bio1_biomass.get_coefficient('cpd00016_c0')
md.reactions.bio1_biomass.add_metabolites({'cpd00104_c0': biotin_coefficient})

##### Purine Metabolism
# Correct GPR for UMP kinase
md.reactions.rxn00119_c0.gene_reaction_rule = 'ADVE02_v2_13049 or ADVE02_v2_14364'
#md.reactions.rxn00119_c0.bounds = (-1000, 1000)


# Gapfill
rxn00363_c0 = cobra.Reaction("rxn00363_c0")
rxn00363_c0.name = 'Cytidine-5-monophosphate phosphohydrolase'
rxn00363_c0.lower_bound = -1000
rxn00363_c0.upper_bound = 1000
rxn00363_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00001_c0'): -1.0,
    md.metabolites.get_by_id('cpd00046_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00009_c0'): 1.0,
    md.metabolites.get_by_id('cpd00367_c0'): 1.0
})
rxn00363_c0.gene_reaction_rule = 'ADVE02_v2_10199' 
md.add_reactions([rxn00363_c0])

rxn01218_c0 = cobra.Reaction("rxn01218_c0")
rxn01218_c0.name = '2-Deoxycytidine-5-monophosphate phosphohydrolase'
rxn01218_c0.lower_bound = -1000
rxn01218_c0.upper_bound = 1000
rxn01218_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00001_c0'): -1.0,
    md.metabolites.get_by_id('cpd00206_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00009_c0'): 1.0,
    md.metabolites.get_by_id('cpd00654_c0'): 1.0
})
rxn01218_c0.gene_reaction_rule = 'ADVE02_v2_10199' 
md.add_reactions([rxn01218_c0])

rxn00708_c0 = cobra.Reaction("rxn00708_c0")
rxn00708_c0.name = 'Uridine-5-monophosphate phosphohydrolase'
rxn00708_c0.lower_bound = -1000
rxn00708_c0.upper_bound = 1000
rxn00708_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00001_c0'): -1.0,
    md.metabolites.get_by_id('cpd00091_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00009_c0'): 1.0,
    md.metabolites.get_by_id('cpd00249_c0'): 1.0
})
rxn00708_c0.gene_reaction_rule = 'ADVE02_v2_10199' 
md.add_reactions([rxn00708_c0])

rxn01145_c0 = cobra.Reaction("rxn01145_c0")
rxn01145_c0.name = 'Thymidylate 5-phosphohydrolase'
rxn01145_c0.lower_bound = -1000
rxn01145_c0.upper_bound = 1000
rxn01145_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00001_c0'): -1.0,
    md.metabolites.get_by_id('cpd00298_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00009_c0'): 1.0,
    md.metabolites.get_by_id('cpd00184_c0'): 1.0
})
rxn01145_c0.gene_reaction_rule = 'ADVE02_v2_10199' 
md.add_reactions([rxn01145_c0])


##### Pyrimidine Metabolism
# Gapfill
rxn00132_c0 = cobra.Reaction("rxn00132_c0")
rxn00132_c0.name = 'Adenosine 5-monophosphate phosphohydrolase'
rxn00132_c0.lower_bound = -1000
rxn00132_c0.upper_bound = 1000
rxn00132_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00001_c0'): -1.0,
    md.metabolites.get_by_id('cpd00018_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00009_c0'): 1.0,
    md.metabolites.get_by_id('cpd00182_c0'): 1.0
})
rxn00132_c0.gene_reaction_rule = 'ADVE02_v2_10199' 
md.add_reactions([rxn00132_c0])

rxn01507_c0 = cobra.Reaction("rxn01507_c0")
rxn01507_c0.name = '2-Deoxyadenosine 5-monophosphate phosphohydrolase'
rxn01507_c0.lower_bound = -1000
rxn01507_c0.upper_bound = 1000
rxn01507_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00001_c0'): -1.0,
    md.metabolites.get_by_id('cpd00294_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00009_c0'): 1.0,
    md.metabolites.get_by_id('cpd00438_c0'): 1.0
})
rxn01507_c0.gene_reaction_rule = 'ADVE02_v2_10199' 
md.add_reactions([rxn01507_c0])

rxn00913_c0 = cobra.Reaction("rxn00913_c0")
rxn00913_c0.name = 'Guanosine-5-monophosphate phosphohydrolase'
rxn00913_c0.lower_bound = -1000
rxn00913_c0.upper_bound = 1000
rxn00913_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00001_c0'): -1.0,
    md.metabolites.get_by_id('cpd00126_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00009_c0'): 1.0,
    md.metabolites.get_by_id('cpd00311_c0'): 1.0
})
rxn00913_c0.gene_reaction_rule = 'ADVE02_v2_10199' 
md.add_reactions([rxn00913_c0])

rxn01445_c0 = cobra.Reaction("rxn01445_c0")
rxn01445_c0.name = '2-Deoxyguanosine-5-monophosphate phosphohydrolase'
rxn01445_c0.lower_bound = -1000
rxn01445_c0.upper_bound = 1000
rxn01445_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00001_c0'): -1.0,
    md.metabolites.get_by_id('cpd00296_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00009_c0'): 1.0,
    md.metabolites.get_by_id('cpd00277_c0'): 1.0
})
rxn01445_c0.gene_reaction_rule = 'ADVE02_v2_10199' 
md.add_reactions([rxn01445_c0])

rxn01961_c0 = cobra.Reaction("rxn01961_c0")
rxn01961_c0.name = 'Xanthosine 5-phosphate phosphohydrolase'
rxn01961_c0.lower_bound = -1000
rxn01961_c0.upper_bound = 1000
rxn01961_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00001_c0'): -1.0,
    md.metabolites.get_by_id('cpd00497_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00009_c0'): 1.0,
    md.metabolites.get_by_id('cpd01217_c0'): 1.0
})
rxn01961_c0.gene_reaction_rule = 'ADVE02_v2_10199' 
md.add_reactions([rxn01961_c0])

rxn00831_c0 = cobra.Reaction("rxn00831_c0")
rxn00831_c0.name = 'Inosine-5-monophosphate phosphohydrolase'
rxn00831_c0.lower_bound = -1000
rxn00831_c0.upper_bound = 1000
rxn00831_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00001_c0'): -1.0,
    md.metabolites.get_by_id('cpd00114_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00009_c0'): 1.0,
    md.metabolites.get_by_id('cpd00246_c0'): 1.0
})
rxn00831_c0.gene_reaction_rule = 'ADVE02_v2_10199' 
md.add_reactions([rxn00831_c0])

##### Nicotinate and nicotinamide metabolism
# Gapfill
rxn01670_c0 = cobra.Reaction("rxn01670_c0")
rxn01670_c0.name = 'Nicotinamide ribonucleotide phosphohydrolase'
rxn01670_c0.lower_bound = -1000
rxn01670_c0.upper_bound = 1000
rxn01670_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00001_c0'): -1.0,
    md.metabolites.get_by_id('cpd00355_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00009_c0'): 1.0,
    md.metabolites.get_by_id('cpd02016_c0'): 1.0
})
rxn01670_c0.gene_reaction_rule = 'ADVE02_v2_10199' 
md.add_reactions([rxn01670_c0])

rxn02400_c0 = cobra.Reaction("rxn02400_c0")
rxn02400_c0.name = 'Nicotinate D-ribonucleotide phosphohydrolase'
rxn02400_c0.lower_bound = -1000
rxn02400_c0.upper_bound = 1000
rxn02400_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00001_c0'): -1.0,
    md.metabolites.get_by_id('cpd00873_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00009_c0'): 1.0,
    md.metabolites.get_by_id('cpd03471_c0'): 1.0
})
rxn02400_c0.gene_reaction_rule = 'ADVE02_v2_10199' 
md.add_reactions([rxn02400_c0])

##### Reactions to be removed
md.remove_reactions(['rxn00058_c0', # Identical to rxn10043 which will be used in this model
                     'rxn09240_c0', # Identical to rxn00379 which will be used in this model
                     'rxn01452_c0', # Identical to rxn01453 which will be used in this model
                     'rxn05144_c0', # Remove pyridxoal phosphate synthesis from G3P+R5P
                     'rxn11938_c0', # Wrong GPR of ADVE02_v2_14359, _14361, _14362: genes are added to rxn02431
                     'rxn01103_c0', # Wrong GPR of ADVE02_v2_20173, _14203: correct rxn already exist, rxn00227
                     'rxn02359_c0', # Wrong GPR of ADVE02_v2_12408: correct rxn already exists, rxn09188
                     'rxn00503_c0', # Wrong GPR of ADVE02_v2_12408: correct rxn already exists, rxn09188
                     'rxn00183_c0', # Wrong GPR of ADVE02_v2_12408: correct rxn already exists, rxn09188
                     'rxn00566_c0', # Wrong GPR of ADVE02_v2_10613: correct rxn already exists, rxn00950
                     'rxn00875_c0', 'rxn04713_c0', # Wrong GPR of ADVE02_v2_11203: correct rxn already exists, rxn00990
                     'rxn10094_c0', # Identical to rxn08130 which will be used in this model
                     'rxn08336_c0', # Assume dihydroorotate oxidase use ubiquinone
                     'rxn08556_c0', 'rxn08557_c0', # Assume glycerol phosphate dehydrogenase does not use ubiquinone/menaquinone
                     'rxn00779_c0', # No genetic evidence found
                     'rxn08647_c0', # No genetic evidence found
                     'rxn01014_c0', # No genetic evidence found
                     'rxn00145_c0', 'rxn08792_c0', 'rxn08793_c0', # Remove lactate/pyruvate oxidoreductase using cytochrome/ubiquinone/menaquinone. rxn01053 is used for lactate/pyruvate oxidoreductase by NADH  
                     'rxn08094_c0', # AKG <-> Succinyl-CoA is demostrated as three-step reaction: rxn00441, rxn02376, rxn01872
                     'rxn00973_c0', # citrate <-> isocitrate is demostrated as two-step reaction: rxn00974, rxn01388
                     'rxn10121_c0', # Assume nitrate reduction to nitrite use NADHP or ubiquinol: shutdown using menaquinol
                     'rxn10806_c0', # Assume cytochrome c oxidase use ubiquinol, remove using menaquinol
                     'rxn08527_c0', 'rxn08528_c0', # Assume succinate/fumarate oxidoreductase use ubiquinol only
                     'rxn00288_c0', 'rxn10126_c0', # Assume succinate/fumarate oxidoreductase as a single-step reaction
                     'rxn00333_c0', # Wrong GPR of ADVE02_v2_14254
                     'rxn08655_c0', # Wrong GPR of ADVE02_v2_14254, _14255, _14256
                     'rxn08656_c0', # Wrong GPR of ADVE02_v2_14254, _14255, _14256
                     'rxn08657_c0', # Wrong GPR of ADVE02_v2_14254, _14255, _14256
                     'rxn00782_c0', # Assume glycerae-1,3PP reduction use NADH, not NADPH
                     'rxn00083_c0', # Assume no oxidoreductase reactions between NADH-NADPH
                     'rxn10125_c0', # Assume no oxidoreductase reactions between NADH-NADPH
                     'rxn09997_c0', # Wrong GPR of ADVE02_v2_14134: correct rxn added, rxn20643_c
                     'rxn00299_c0', # GTP -> dihydroneopterin is demonstrated as four-step reaction: rxn00302, rxn03419, rxn03421, rxn03174
                     'rxn05733_c0', # Identical to rxn00679 which will be used in this model
                     'rxn03012_c0', # Identical to rxn10484 which will be used in this model
                     'rxn08086_c0', 'rxn08549_c0', 'rxn07579_c0', 'rxn05744_c0', 'rxn09436_c0', 'rxn08088_c0', 'rxn08551_c0', 'rxn10954_c0', # FA synthesis redundant reactions
                     'rxn01280_c0', 'rxn01281_c0', # Wrong GPR for ADVE02_v2_10838: correctly assigned to rxn01480
                     'rxn01870_c0', # Wrong GPR of ADVE02_v2_11247: correct rxns already exist, rxn00786, rxn01334 
                     'rxn00199_c0', 'rxn01387_c0', # Wrong GPR of ADVE02_v2_10351: correctly assigned to rxn00198
                     'rxn00543_c0', # Wrong GPR of ADVE02_v2_10696, ADVE02_v2_20074, ADVE02_v2_20075: correct rxn already exists, rxn00763
                     'rxn03406_c0', 'rxn03407_c0', 'rxn03409_c0', # Wrong GPR of ADVE02_v2_10779: correct rxn already exists, rxn00187
                     'rxn03933_c0', # Wrong GPR of ADVE02_v2_10774: correct rxn already exists, rxn03408
                     'rxn10194_c0', 'rxn10195_c0', 'rxn10196_c0', # involved in teichoic acid metabolism, which is found in gram-positive cells
                     'rxn06673_c0', 'rxn06672_c0', 'rxn00672_c0', # Wrong GPR, correctly assigned to rxn01355, rxn00533
                     'rxn00707_c0', # Wrong GPR, correct rxns already exist, rxn00364, rxn01219, rxn00119
                     'rxn03903_c0', # Wrong GPR

                     # Exchange/Transport reactions
                     'EX_cpd00013_e0', 'rxn05466_c0', # NH3
                     'EX_cpd00075_e0', 'rxn05625_c0', # NO2
                     'EX_cpd00268_e0', 'rxn05176_c0', # Thiosulfate
                     'EX_cpd00081_e0', 'rxn12453_c0', # Sulfite
                     
                     
                     'EX_cpd00023_e0', 'rxn05298_c0', # L-Glutamate                     
                     'EX_cpd00041_e0', 'rxn05152_c0', # L-Aspartate
                     'EX_cpd00051_e0', 'rxn05154_c0', # L-Arginine
                     'EX_cpd00107_e0', 'rxn05161_c0', # L-Leucine
                     'EX_cpd00129_e0', 'rxn05165_c0', # L-Proline
                     'EX_cpd00130_e0', 'rxn05605_c0', # L-Malate
                     'EX_cpd00156_e0', 'rxn05168_c0', # L-Valine
                     'EX_cpd00322_e0', 'rxn05179_c0', # L-Isoleucine
                     'EX_cpd00395_e0', 'rxn05599_c0', # L-Cysteate
                     
                     
                     'EX_cpd00182_e0', 'rxn05318_c0', # Adenosine
                     'EX_cpd00184_e0', 'rxn05200_c0', # Thymidine
                     'EX_cpd00246_e0', 'rxn05316_c0', # Inosine
                     'EX_cpd00249_e0', 'rxn05199_c0', # Uridine
                     'EX_cpd00277_e0', 'rxn09688_c0', # Deoxyguanosine
                     'EX_cpd00305_e0', 'rxn09657_c0', # Thiamin
                     'EX_cpd00367_e0', 'rxn05198_c0', # Cytidine
                     'EX_cpd00412_e0', 'rxn09685_c0', # Deoxyuridine
                     'EX_cpd00438_e0', 'rxn05317_c0', # Deoxyadenosine
                     'EX_cpd00654_e0', 'rxn05205_c0', # Deoxycytidine
                     'EX_cpd03279_e0', 'rxn09687_c0', # Deoxyinosine
                     
                     
                     #'EX_cpd00118_e0', 'rxn05163_c0', # Putrescine
                     'EX_cpd00393_e0', 'rxn05255_c0', # Folate
                     'EX_cpd00106_e0', 'rxn05561_c0', # Fumarate
                     'EX_cpd00024_e0', 'rxn05493_c0', # AKG
                     'EX_cpd00036_e0', 'rxn05654_c0', # Succinate
                     'EX_cpd00540_e0', 'rxn05181_c0', # Glycin betaine
                     'EX_cpd00644_e0', 'rxn10180_c0', # Pantothenate
                     'EX_cpd00210_e0', 'rxn05172_c0', # Taurine
                     'EX_cpd00215_e0', 'rxn12666_c0', # Pyridoxal
                     'EX_cpd00220_e0', 'rxn05645_c0', # Riboflavin
                     'EX_cpd00264_e0', 'rxn05175_c0', # Spermidine
                     'EX_cpd00635_e0', 'rxn10147_c0', # Vitamin B12s, Cbl-I
                     'EX_cpd00423_e0', 'rxn05515_c0', # Vitamin B12r, Cbl-II
                     'EX_cpd03424_e0', 'rxn05187_c0', # Vitamin B12, Cbl-III
                     'EX_cpd01015_e0', 'rxn10132_c0', # Cyanate
                     'EX_cpd03048_e0', 'rxn05594_c0', # Isethionate
                     'EX_cpd03343_e0', 'rxn10133_c0', # Phenylpropanoate
                     'EX_cpd08023_e0', 'rxn05621_c0', # Methanesulfonate
                     'EX_cpd09878_e0', 'rxn05656_c0', # Sulfoacetate
                     'EX_cpd11575_e0', 'rxn05620_c0', # MOPS
                     'EX_cpd11578_e0', 'rxn05585_c0', # Hexanesulfonate
                     'EX_cpd11579_e0', 'rxn05552_c0', # Ethanesulfonate
                     'EX_cpd11596_e0', 'rxn05512_c0', # Butanesulfonate
                     
                     
                     'EX_cpd00531_e0', 'rxn10343_c0', # Hg2+
                     'EX_cpd01012_e0', 'rxn05516_c0', # Cd2+
                     'EX_cpd01048_e0', 'rxn05506_c0', # Arsenate
                     'EX_cpd04097_e0', 'rxn10344_c0', # Pb
                     'EX_cpd04098_e0', 'rxn05507_c0', # Arsenite
                     
                     
                     'EX_cpd01017_e0', 'rxn05539_c0', 'rxn00650_c0', # Cys-Gly
                     'EX_cpd11580_e0', 'rxn05547_c0', 'rxn12646_c0', # Gly-Gln
                     'EX_cpd11581_e0', 'rxn05546_c0', 'rxn12645_c0', # Gly-Asn
                     'EX_cpd11582_e0', 'rxn05545_c0', 'rxn12644_c0', # Ala-Thr
                     'EX_cpd11583_e0', 'rxn05544_c0', 'rxn12643_c0', # Ala-Leu
                     'EX_cpd11584_e0', 'rxn05543_c0', 'rxn12642_c0', # Ala-His
                     'EX_cpd11585_e0', 'rxn05542_c0', 'rxn12641_c0', # Ala-Gly
                     'EX_cpd11586_e0', 'rxn05541_c0', 'rxn12640_c0', # Ala-Glu
                     'EX_cpd11587_e0', 'rxn05540_c0', 'rxn12639_c0', # Ala-Gln
                     'EX_cpd11588_e0', 'rxn05538_c0', 'rxn12638_c0', # Gly-Pro
                     'EX_cpd11589_e0', 'rxn05537_c0', 'rxn12637_c0', # Gly-Asp
                     'EX_cpd11590_e0', 'rxn05536_c0', 'rxn12636_c0', # Met-Ala
                     'EX_cpd11591_e0', 'rxn05535_c0', 'rxn12635_c0', # Gly-Met
                     'EX_cpd11592_e0', 'rxn05534_c0', 'rxn12634_c0', # Gly-Glu
                     'EX_cpd11593_e0', 'rxn05533_c0', 'rxn12633_c0', # Ala-Asp
                     'EX_cpd15603_e0', 'rxn12848_c0', 'rxn12844_c0', # Gly-Cys
                     'EX_cpd15604_e0', 'rxn12849_c0', 'rxn12845_c0', # Gly-Leu
                     'EX_cpd15605_e0', 'rxn12850_c0', 'rxn12846_c0', # Gly-Phe
                     'EX_cpd15606_e0', 'rxn12851_c0', 'rxn12847_c0' # Gly-Tyr
                    ], remove_orphans = True)




##### Biomass

uniquinone_coefficient = md.reactions.bio1_biomass.get_coefficient('cpd15560_c0')
md.reactions.bio1_biomass.add_metabolites({'cpd15560_c0':-uniquinone_coefficient})
md.reactions.bio1_biomass.add_metabolites({'cpd15561_c0':uniquinone_coefficient})

menaquinone_coefficient = md.reactions.bio1_biomass.get_coefficient('cpd15500_c0')
md.reactions.bio1_biomass.add_metabolites({'cpd15500_c0':-menaquinone_coefficient})
md.reactions.bio1_biomass.add_metabolites({'cpd15499_c0':menaquinone_coefficient})

demethylmenaquinone_coefficient = md.reactions.bio1_biomass.get_coefficient('cpd15352_c0')
md.reactions.bio1_biomass.add_metabolites({'cpd15352_c0':-demethylmenaquinone_coefficient})
md.reactions.bio1_biomass.add_metabolites({'cpd15353_c0':demethylmenaquinone_coefficient})


##### Environmental condition: no cerium
md.reactions.rxn00847_c0.bounds = (0,0) # Shutdowon methanol oxidation using pqq


# cobra.io.save_json_model(md, ModelDir + "GEM_ADVE_v10d.json")
cobra.io.write_sbml_model(md, ModelDir + "GEM_ADVE_v10d.xml")



In [ ]:
md = cobra.io.read_sbml_model(ModelDir + 'GEM_ADVE_v10d.xml')

# Add PHB in biomass, and PHB sythesis/degradation reactions
phb_c0 = cobra.Metabolite('phb_c0', formula='PHB', name='PHB', compartment='c0')
md.reactions.bio1_biomass.add_metabolites({phb_c0:-0.242483}) # Based on PHB consists ~2.5% of gDCW

cpd00797_c0 = cobra.Metabolite('cpd00797_c0', formula='C4H7O3', name='(R)-3-Hydroxybutanoate_c0', compartment='c0')
# PHB synthesis
phb_polymerase_c0 = cobra.Reaction("phb_polymerase_c0")
phb_polymerase_c0.name = 'PHB_polymerase'
phb_polymerase_c0.lower_bound = 0
phb_polymerase_c0.upper_bound = 1000
phb_polymerase_c0.add_metabolites({
    md.metabolites.get_by_id('cpd02234_c0'): -1.0,

    md.metabolites.get_by_id('cpd00010_c0'): 1.0,
    phb_c0: 1.0
})
phb_polymerase_c0.gene_reaction_rule = 'ADVE02_v2_14145 or ADVE02_v2_12833'
md.add_reactions([phb_polymerase_c0])

# # PHB accumulation expressed as sink reaction
# phb_accumulation_c0 = cobra.Reaction("phb_accumulation_c0")
# phb_accumulation_c0.name = 'PHB_accumulation'
# phb_accumulation_c0.lower_bound = 0.1
# phb_accumulation_c0.upper_bound = 1000
# phb_accumulation_c0.add_metabolites({
#     phb_c0: -1.0
# })
# md.add_reactions([phb_accumulation_c0])

# PHB degradation
phb_depolymerase_c0 = cobra.Reaction("phb_depolymerase_c0")
phb_depolymerase_c0.name = 'PHB_depolymerase'
phb_depolymerase_c0.lower_bound = 0
phb_depolymerase_c0.upper_bound = 0
phb_depolymerase_c0.add_metabolites({
    phb_c0: -1.0,
    md.metabolites.get_by_id('cpd00001_c0'): -1.0,

    cpd00797_c0: 1.0
})
phb_depolymerase_c0.gene_reaction_rule = 'ADVE02_v2_11620 or ADVE02_v2_13857 or ADVE02_v2_13998'
md.add_reactions([phb_depolymerase_c0])


# # PHB degradation
# rxn00992_c0 = cobra.Reaction("rxn00992_c0")
# rxn00992_c0.name = '(R)-3-Hydroxybutanoate:NAD+ oxidoreductase'
# rxn00992_c0.lower_bound = 0
# rxn00992_c0.upper_bound = 1000
# rxn00992_c0.add_metabolites({
#     md.metabolites.get_by_id('cpd00003_c0'): -1.0,
#     cpd00797_c0: -1.0,
    
#     md.metabolites.get_by_id('cpd00004_c0'): 1.0,
#     md.metabolites.get_by_id('cpd00067_c0'): 1.0,
#     md.metabolites.get_by_id('cpd00142_c0'): 1.0
# })
# rxn00992_c0.gene_reaction_rule = 'ADVE02_v2_14456'
# md.add_reactions([rxn00992_c0])
md.reactions.rxn00992_c0.bounds = (0,0)


# PHB degradation
rxn00290_c0 = cobra.Reaction("rxn00290_c0")
rxn00290_c0.name = '(R)-3-Hydroxybutanoate:NAD+ oxidoreductase'
rxn00290_c0.lower_bound = 0
rxn00290_c0.upper_bound = 0
rxn00290_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00078_c0'): -1.0,
    md.metabolites.get_by_id('cpd00142_c0'): -1.0,
    
    md.metabolites.get_by_id('cpd00036_c0'): 1.0,
    md.metabolites.get_by_id('cpd00279_c0'): 1.0
})
rxn00290_c0.gene_reaction_rule = 'ADVE02_v2_11203'
md.add_reactions([rxn00290_c0])

# AKG -> Succinyl-CoA
rxn08094_c0 = cobra.Reaction("rxn08094_c0")
rxn08094_c0.name = '2-Oxoglutarate dehydrogenase complex'
rxn08094_c0.lower_bound = 0
rxn08094_c0.upper_bound = 1000
rxn08094_c0.add_metabolites({
    md.metabolites.get_by_id('cpd00003_c0'): -1.0,
    md.metabolites.get_by_id('cpd00010_c0'): -1.0,
    md.metabolites.get_by_id('cpd00024_c0'): -1.0,
    
    
    md.metabolites.get_by_id('cpd00004_c0'): 1.0,
    md.metabolites.get_by_id('cpd00011_c0'): 1.0,
    md.metabolites.get_by_id('cpd00078_c0'): 1.0
})
rxn08094_c0.gene_reaction_rule = 'ADVE02_v2_11677 and ADVE02_v2_11678 and ADVE02_v2_11679'
md.add_reactions([rxn08094_c0])

# Spermidine is not detected in wild-type cells. Remove from biomass reaction
spermidine_coefficient = md.reactions.bio1_biomass.get_coefficient('cpd00264_c0')
md.reactions.bio1_biomass.add_metabolites({'cpd00264_c0':-spermidine_coefficient})

# md.reactions.rxn00154_c0.bounds = (0.4, 1000)
md.reactions.rxn00154_c0.gene_reaction_rule = 'ADVE02_v2_10246 and ADVE02_v2_10248 and ADVE02_v2_10249 and ADVE02_v2_13740 and ADVE02_v2_13741 and ADVE02_v2_13743'
md.reactions.rxn00154_c0.bounds = (0, 1000)

md.reactions.rxn00992_c0.bounds = (0, 1000)

md.reactions.rxn00929_c0.bounds = (-1000,0)
md.reactions.rxn00931_c0.bounds = (-1000,0)
md.reactions.rxn00467_c0.bounds = (0,1000)

md.reactions.rxn00178_c0.bounds = (0,1000)
md.reactions.rxn00990_c0.bounds = (-1000,0)

md.reactions.rxn00175_c0.bounds = (0,1000)
md.reactions.rxn00225_c0.bounds = (0,1000)
md.reactions.NGAM_c0.bounds = (15,15)

md.reactions.rxn00331_c0.gene_reaction_rule = 'ADVE02_v2_11719 or ADVE02_v2_12560'


# # Add putrescine exchange reaction
# Putrescine_exchange = cobra.Reaction("Putrescine_exchange")
# Putrescine_exchange.name = 'Putrescine_exchange'
# Putrescine_exchange.lower_bound = 0
# Putrescine_exchange.upper_bound = 1000
# Putrescine_exchange.add_metabolites({
#     md.metabolites.get_by_id('cpd00118_c0'): -1.0
# })
# md.add_reactions([Putrescine_exchange])


md.reactions.rxn00260_c0.gene_reaction_rule = 'ADVE02_v2_12831 or ADVE02_v2_10599 or ADVE02_v2_10391 or (ADVE02_v2_12112 and ADVE02_v2_12113)'


md.remove_reactions(['rxn00441_c0',
                     'rxn01871_c0',
                     'rxn01872_c0',
                     'rxn01925_c0',
                     'rxn02342_c0',
                     'rxn02376_c0',
                     'rxn07430_c0',
                     'rxn07431_c0',
                     'rxn00904_c0',
                     'rxn00990_c0'
                    ], remove_orphans = True)





cobra.io.write_sbml_model(md, ModelDir + "GEM_ADVE_OB3b_sMMO.xml")
# cobra.io.save_json_model(md, ModelDir + "GEM_ADVE_OB3b_sMMO.json")



In [ ]:
md = cobra.io.read_sbml_model(ModelDir + 'GEM_ADVE_OB3b_sMMO.xml')

### Add pMMO reactions

# pMMO for Direct-coupling mode
pMMO_DirectCoupling_rxn = cobra.Reaction("pMMO_DC_c0")
pMMO_DirectCoupling_rxn.name = 'particulate methane monooxygenase: direct-coupling mode'
pMMO_DirectCoupling_rxn.lower_bound = 0
pMMO_DirectCoupling_rxn.upper_bound = 1000
pMMO_DirectCoupling_rxn.add_metabolites({
    md.metabolites.get_by_id('cpd01024_c0'): -1.0,
    md.metabolites.get_by_id('cpd00110_c0'): -2.0,
    md.metabolites.get_by_id('cpd00007_c0'): -1.0,
    md.metabolites.get_by_id('cpd00067_c0'): -2.0,

    md.metabolites.get_by_id('cpd00116_c0'): 1.0,
    md.metabolites.get_by_id('cpd00109_c0'): 2.0,
    md.metabolites.get_by_id('cpd00001_c0'): 1.0
})
pMMO_DirectCoupling_rxn.gene_reaction_rule = '(ADVE02_v2_10402 or ADVE02_12978) and (ADVE02_v2_10400 or ADVE02_12979) and (ADVE02_v2_10399 or ADVE02_12980)'
md.add_reactions([pMMO_DirectCoupling_rxn])

# pMMO for Redox-Arm and Uphill-e-Transfer modes
pMMO_rxn = cobra.Reaction("pMMO_c0")
pMMO_rxn.name = 'particulate methane monooxygenase'
pMMO_rxn.lower_bound = 0
pMMO_rxn.upper_bound = 1000
pMMO_rxn.add_metabolites({
    md.metabolites.get_by_id('cpd01024_c0'): -1.0,
    md.metabolites.get_by_id('cpd00007_c0'): -1.0,
    md.metabolites.get_by_id('cpd15561_c0'): -1.0,

    md.metabolites.get_by_id('cpd00116_c0'): 1.0,
    md.metabolites.get_by_id('cpd15560_c0'): 1.0,
    md.metabolites.get_by_id('cpd00001_c0'): 1.0
})
pMMO_rxn.gene_reaction_rule = '(ADVE02_v2_10402 or ADVE02_12978) and (ADVE02_v2_10400 or ADVE02_12979) and (ADVE02_v2_10399 or ADVE02_12980)'
md.add_reactions([pMMO_rxn])


md.reactions.rxn13689_c0.bounds = (-1000,1000)

# Shutdown sMMO
# v13d.reactions.rxn00843_c0.bounds = (0, 0)
md.remove_reactions(['rxn00843_c0'], remove_orphans = True)

# cobra.io.save_json_model(md, ModelDir + "GEM_ADVE_OB3b_pMMO.json")
cobra.io.write_sbml_model(md, ModelDir + "GEM_ADVE_OB3b_pMMO.xml")

In [ ]:
# Direct-Coupling mode

pmmo_dc = cobra.io.read_sbml_model(ModelDir + 'GEM_ADVE_OB3b_pMMO.xml')

# Shutdown redox-arm and uphill-e-transfer modes
pmmo_dc.remove_reactions(['pMMO_c0'], remove_orphans = True)

# Activate pMMO direct-coupling mode
pmmo_dc.reactions.pMMO_DC_c0.bounds = (0,1000)

# ubiquinol-cytochrome oxidoreductase is irreversible for direct-coupling and redox-arm modes
pmmo_dc.reactions.rxn13689_c0.bounds = (0,1000)

# cobra.io.save_json_model(pmmo_dc, ModelDir + "GEM_ADVE_OB3b_pMMO_DC.json")
cobra.io.write_sbml_model(pmmo_dc, ModelDir + "GEM_ADVE_OB3b_pMMO_DC.xml")

In [ ]:
# Redox-Arm mode

pmmo_ra = cobra.io.read_sbml_model(ModelDir + 'GEM_ADVE_OB3b_pMMO.xml')

# Shutdown direct-coupling mode
pmmo_ra.remove_reactions(['pMMO_DC_c0'], remove_orphans = True)

# Activate pMMO redox-arm mode
pmmo_ra.reactions.pMMO_c0.bounds = (0,1000)

# ubiquinol-cytochrome oxidoreductase is irreversible for direct-coupling and redox-arm modes
pmmo_ra.reactions.rxn13689_c0.bounds = (0,1000)

# cobra.io.save_json_model(pmmo_ra, ModelDir + "GEM_ADVE_OB3b_pMMO_RA.json")
cobra.io.write_sbml_model(pmmo_ra, ModelDir + "GEM_ADVE_OB3b_pMMO_RA.xml")

In [ ]:
# Uphill-e-Transfer mode

pmmo_ut = cobra.io.read_sbml_model(ModelDir + 'GEM_ADVE_OB3b_pMMO.xml')

# Shutdown direct-coupling mode
pmmo_ut.remove_reactions(['pMMO_DC_c0'], remove_orphans = True)

# Activate pMMO Uphill-e-Transfer mode
pmmo_ut.reactions.pMMO_c0.bounds = (0,1000)

# ubiquinol-cytochrome oxidoreductase is reversible for uphill-e-transfer mode
pmmo_ut.reactions.rxn13689_c0.bounds = (-1000,1000)

# cobra.io.save_json_model(pmmo_ut, ModelDir + "GEM_ADVE_OB3b_pMMO_UT.json")
cobra.io.write_sbml_model(pmmo_ut, ModelDir + "GEM_ADVE_OB3b_pMMO_UT.xml")